In [3]:
!pip install -q underthesea

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.9/20.9 MB 71.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 657.8/657.8 kB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 35.2 MB/s eta 0:00:00:00:01


In [5]:
!git clone https://github.com/stopwords/vietnamese-stopwords

Cloning into 'vietnamese-stopwords'...
remote: Enumerating objects: 95, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 95 (delta 3), reused 0 (delta 0), pack-reused 81 (from 1)
Receiving objects: 100% (95/95), 40.25 KiB | 3.10 MiB/s, done.
Resolving deltas: 100% (31/31), done.


In [7]:
with open("/kaggle/working/vietnamese-stopwords/vietnamese-stopwords.txt", "r", encoding="utf-8") as f:
    vietnamese_stop_words = set(f.read().splitlines())

In [6]:
import numpy as np
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, accuracy_score
from underthesea import sentiment, ner

In [17]:
file_path="/kaggle/input/vietnamese-text-classification-dataset/train.csv"
df=pd.read_csv(file_path)
df

,0,máy dùng hay bị đơ máy
0,0,chỉ có dây cáp nguồn không có adapter sao sử d...
1,0,Chất lượng quá kém Mới dùng được 2 ngày loa ba...
2,0,Usb tôi vừa mới nhận usb này Rất bực bội vì cá...
3,2,Tuyệt vời. Hàng FPT cửa hàng
4,1,Chưa đo đúng bước chân rõ Nhiều khi gián đoạn ...
...,...,...
3034,0,không có bộ thu đi kèm Vừa nhận hàng xong tức ...
3035,0,Cũng bình thường Không mạnh như kỹ vọng Chắc t...
3036,2,tốt tốt đang tiền khi bỏ ra để mua giao hàng n...
3037,0,giao hàng lâu rửa tay một tí đã cháy màn hình ...


In [18]:
df.columns = ["label","text"]
df

,label,text
0,0,chỉ có dây cáp nguồn không có adapter sao sử d...
1,0,Chất lượng quá kém Mới dùng được 2 ngày loa ba...
2,0,Usb tôi vừa mới nhận usb này Rất bực bội vì cá...
3,2,Tuyệt vời. Hàng FPT cửa hàng
4,1,Chưa đo đúng bước chân rõ Nhiều khi gián đoạn ...
...,...,...
3034,0,không có bộ thu đi kèm Vừa nhận hàng xong tức ...
3035,0,Cũng bình thường Không mạnh như kỹ vọng Chắc t...
3036,2,tốt tốt đang tiền khi bỏ ra để mua giao hàng n...
3037,0,giao hàng lâu rửa tay một tí đã cháy màn hình ...


# Sentiment Analysis

In [57]:
from underthesea import sentiment

tests = [
    "Sản phẩm này thật tuyệt vời!",
    "Tôi rất hài lòng về chất lượng dịch vụ.",
    "Thức ăn ngon, nhân viên phục vụ nhiệt tình.",
    "Trải nghiệm thật sự đáng nhớ, tôi sẽ quay lại.",
    "Giá rẻ mà chất lượng tốt bất ngờ.",

    "Dịch vụ ở đây quá tệ.",
    "Tôi thất vọng về sản phẩm này.",
    "Giao hàng chậm, nhân viên thô lỗ.",
    "Chất lượng quá kém, không đáng tiền.",
    "Tôi sẽ không bao giờ quay lại nữa.",

    "Hôm nay trời nắng.",
    "Tôi đi siêu thị mua ít đồ.",
    "Cuốn sách này có 300 trang.",
    "Anh ấy sống ở Hà Nội.",
    "Tôi có lịch họp vào lúc 9 giờ sáng.",

    "Dịch vụ ở quán này không tệ.",
    "Sản phẩm này không hề rẻ chút nào.",
    "Tôi không ghét món ăn này.",
    "Bộ phim không phải dở, nhưng cũng không hay.",
    "Thái độ của nhân viên không tốt lắm."
]

for t in tests:
    print(f"{t} --> {sentiment(t)}")


Sản phẩm này thật tuyệt vời! --> positive
Tôi rất hài lòng về chất lượng dịch vụ. --> positive
Thức ăn ngon, nhân viên phục vụ nhiệt tình. --> positive
Trải nghiệm thật sự đáng nhớ, tôi sẽ quay lại. --> positive
Giá rẻ mà chất lượng tốt bất ngờ. --> positive
Dịch vụ ở đây quá tệ. --> negative
Tôi thất vọng về sản phẩm này. --> negative
Giao hàng chậm, nhân viên thô lỗ. --> negative
Chất lượng quá kém, không đáng tiền. --> negative
Tôi sẽ không bao giờ quay lại nữa. --> negative
Hôm nay trời nắng. --> positive
Tôi đi siêu thị mua ít đồ. --> negative
Cuốn sách này có 300 trang. --> None
Anh ấy sống ở Hà Nội. --> None
Tôi có lịch họp vào lúc 9 giờ sáng. --> None
Dịch vụ ở quán này không tệ. --> negative
Sản phẩm này không hề rẻ chút nào. --> positive
Tôi không ghét món ăn này. --> negative
Bộ phim không phải dở, nhưng cũng không hay. --> negative
Thái độ của nhân viên không tốt lắm. --> negative


# Named Entities Recognition

In [4]:
from underthesea import ner

texts = [
    "Ông Tô Lâm là Tổng bí thư Đảng Cộng sản Việt Nam.",
    "Lan làm việc tại Công ty FPT ở Hà Nội."
]

for t in texts:
    print(f"Input: {t}")
    print("NER:", ner(t))
    print("-" * 80)

Input: Ông Tô Lâm là Tổng bí thư Đảng Cộng sản Việt Nam.
NER: [('Ông', 'Nc', 'B-NP', 'O'), ('Tô Lâm', 'Np', 'B-NP', 'B-PER'), ('là', 'V', 'B-VP', 'O'), ('Tổng bí thư', 'N', 'B-NP', 'O'), ('Đảng Cộng sản Việt Nam', 'V', 'B-VP', 'O'), ('.', 'CH', 'O', 'O')]
--------------------------------------------------------------------------------
Input: Lan làm việc tại Công ty FPT ở Hà Nội.
NER: [('Lan', 'Np', 'B-NP', 'B-PER'), ('làm việc', 'V', 'B-VP', 'O'), ('tại', 'E', 'B-PP', 'O'), ('Công ty', 'Np', 'B-NP', 'B-LOC'), ('FPT', 'Np', 'I-NP', 'I-LOC'), ('ở', 'E', 'B-PP', 'O'), ('Hà Nội', 'Np', 'B-NP', 'B-LOC'), ('.', 'CH', 'O', 'O')]
--------------------------------------------------------------------------------


# Text classification

In [36]:
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from typing import List, Dict, Any, Optional, Callable

class TextPreprocessor:
    def __init__(self, stop_words: Optional[List[str]] = None):
        self.stop_words = stop_words or []
    
    def basic_clean(self, text: str) -> str:
        """Basic text cleaning without stopword removal"""
        text = str(text).lower()
        text = re.sub(r'http\S+|www\.\S+', '', text)  
        text = re.sub(r'[^0-9a-zA-ZÀ-Ỵà-ỵ\s]', ' ', text) 
        text = re.sub(r'\s+', ' ', text)
        text = re.sub(r'[^a-zA-ZÀ-Ỵà-ỵ\s]', ' ', text)
        return text.strip()
    
    def clean_with_stopwords(self, text: str) -> str:
        """Text cleaning with stopword removal"""
        text = self.basic_clean(text)
        words = text.split()
        words = [w for w in words if w not in self.stop_words]
        return " ".join(words)
    

class TextClassificationPipeline:
    
    def __init__(self):
        self.preprocessor = None
        self.vectorizer = None
        self.model = None
        self.preprocessing_func = None
    
    def setup_preprocessing(self, method: Callable[[str], str], stop_words: Optional[List[str]] = None):
        self.preprocessor = TextPreprocessor(stop_words)
        if isinstance(method, str):
            if not hasattr(self.preprocessor, method):
                raise ValueError(f"Preprocessor has no method {method}")
            self.preprocessing_func = getattr(self.preprocessor, method)
        elif callable(method):
            self.preprocessing_func = method
        else:
            raise ValueError("method must be string or callable")
    
    def setup_vectorizer(self, vectorizer: Any):
        self.vectorizer = vectorizer
    
    def setup_model(self, model: Any):
        self.model = model
    
    def preprocess_data(self, df: pd.DataFrame, text_column: str = 'text') -> pd.DataFrame:
        if self.preprocessing_func is None:
            raise ValueError("Preprocessing method not set. Call setup_preprocessing() first.")
        
        df_copy = df.copy()
        df_copy['clean_text'] = df_copy[text_column].apply(self.preprocessing_func)
        return df_copy
    
    def train_and_evaluate(self, 
                          df: pd.DataFrame, 
                          text_column: str = 'clean_text',
                          label_column: str = 'label',
                          test_size: float = 0.2,
                          random_state: int = 42) -> Dict[str, Any]:
        """Train model and return evaluation results"""
        if self.vectorizer is None or self.model is None:
            raise ValueError("Vectorizer and model must be set up first.")
        
        X = df[text_column]
        y = df[label_column]
    
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=test_size, random_state=random_state, stratify=y
        )
        
        X_train_vec = self.vectorizer.fit_transform(X_train)
        X_test_vec = self.vectorizer.transform(X_test)
        
        self.model.fit(X_train_vec, y_train)
        y_pred = self.model.predict(X_test_vec)
        
        accuracy = accuracy_score(y_test, y_pred)
        report = classification_report(y_test, y_pred, output_dict=True)
        
        return {
            'accuracy': accuracy,
            'classification_report': report,
            'y_true': y_test,
            'y_pred': y_pred,
            'model': self.model,
            'vectorizer': self.vectorizer
        }


def run_experiment(df: pd.DataFrame, 
                  preprocessing_method: str,
                  vectorizer: Any,
                  model: Any,
                  stop_words: Optional[List[str]] = None) -> Dict[str, Any]:
    """Run a single experiment with specified configuration"""
    
    pipeline = TextClassificationPipeline()
    pipeline.setup_preprocessing(preprocessing_method, stop_words)
    pipeline.setup_vectorizer(vectorizer)
    pipeline.setup_model(model)
    
    df_processed = pipeline.preprocess_data(df)
    results = pipeline.train_and_evaluate(df_processed)
    
    results['config'] = {
        'preprocessing': preprocessing_method,
        'vectorizer': str(vectorizer),
        'model': str(model),
    }
    
    return results


if __name__ == "__main__":
    ...


In [48]:
result = run_experiment(
    df=df,
    preprocessing_method='basic_clean',
    vectorizer=TfidfVectorizer(),
    model=LinearSVC(),
)

print(f"Accuracy: {result['accuracy']:.4f}")
print(classification_report(result['y_true'], result['y_pred']))

Accuracy: 0.7780
              precision    recall  f1-score   support

           0       0.78      0.86      0.81       221
           1       0.68      0.61      0.64       177
           2       0.85      0.84      0.85       210

    accuracy                           0.78       608
   macro avg       0.77      0.77      0.77       608
weighted avg       0.78      0.78      0.78       608



In [50]:
result = run_experiment(
    df=df,
    preprocessing_method='clean_with_stopwords',
    vectorizer=TfidfVectorizer(),
    model=LinearSVC(),
    stop_words=vietnamese_stop_words
)
print(f"Accuracy: {result['accuracy']:.4f}")
print(classification_report(result['y_true'], result['y_pred']))

Accuracy: 0.7007
              precision    recall  f1-score   support

           0       0.69      0.72      0.70       221
           1       0.62      0.55      0.58       177
           2       0.78      0.80      0.79       210

    accuracy                           0.70       608
   macro avg       0.69      0.69      0.69       608
weighted avg       0.70      0.70      0.70       608



In [52]:
result = run_experiment(
    df=df,
    preprocessing_method='basic_clean',
    vectorizer=TfidfVectorizer(),
    model=MultinomialNB(),
)

print(f"Accuracy: {result['accuracy']:.4f}")
print(classification_report(result['y_true'], result['y_pred']))

Accuracy: 0.7352
              precision    recall  f1-score   support

           0       0.70      0.83      0.76       221
           1       0.67      0.47      0.56       177
           2       0.81      0.85      0.83       210

    accuracy                           0.74       608
   macro avg       0.73      0.72      0.72       608
weighted avg       0.73      0.74      0.73       608



In [54]:
result = run_experiment(
    df=df,
    preprocessing_method='clean_with_stopwords',
    vectorizer=TfidfVectorizer(),
    model=MultinomialNB(),
    stop_words=vietnamese_stop_words
)

print(f"Accuracy: {result['accuracy']:.4f}")
print(classification_report(result['y_true'], result['y_pred']))

Accuracy: 0.6974
              precision    recall  f1-score   support

           0       0.69      0.77      0.73       221
           1       0.65      0.43      0.52       177
           2       0.72      0.85      0.78       210

    accuracy                           0.70       608
   macro avg       0.69      0.68      0.68       608
weighted avg       0.69      0.70      0.69       608

